# Контекстно-свободные грамматики  
Курс NLP, семинар 11.  
Осень 2017.

Подпись: Калитова Ольга Сергеевна АД(?) 4(?)

![](http://i.imgur.com/XLREeKo.png)

## Немного теории

*Формальная грамматика* (определение с лекции):
 - конечное множество нетерминалов $P$;
 - конечное множество терминалов $\Sigma$;
 - конечное множество продукций $P$, каждая из которых имеет вид
 
$$(\Sigma \cup N)^∗ N (\Sigma \cup N)^∗ \rightarrow (\Sigma \cup N)^∗$$
 - начальный нетерминал $S$.
 
*Контекстно-свободная грамматика* имеет правила вида $A \rightarrow \beta, \beta \in (N \cup \Sigma)^*$.


Контекстно-свободная является грамматикой в *нормальной форме Хомского*, если содержит только правила вида: 
 - $A \rightarrow B C$
 - $A \rightarrow a$
 - $S \rightarrow \varepsilon$

где $a$ $-$ терминал, $A, B, C$ $-$ нетерминалы, $S$ $-$ начальный нетерминал не содержащийся в правых частях правил, $\varepsilon$ $-$ пустая строка.

Как привести грамматику к нормальной форме Хомского можно прочитать [здесь](http://math.stackexchange.com/questions/296243/converting-to-chomsky-normal-form) или [здесь](http://neerc.ifmo.ru/wiki/index.php?title=Нормальная_форма_Хомского#.D0.9F.D1.80.D0.B8.D0.B2.D0.B5.D0.B4.D0.B5.D0.BD.D0.B8.D0.B5_.D0.B3.D1.80.D0.B0.D0.BC.D0.BC.D0.B0.D1.82.D0.B8.D0.BA.D0.B8_.D0.BA_.D0.BD.D0.BE.D1.80.D0.BC.D0.B0.D0.BB.D1.8C.D0.BD.D0.BE.D0.B9_.D1.84.D0.BE.D1.80.D0.BC.D0.B5_.D0.A5.D0.BE.D0.BC.D1.81.D0.BA.D0.BE.D0.B3.D0.BE).

## 1. Chomsky Normal Form 
Приведите грамматику к нормальной форме Хомского (и выпишите итоговую <font color='red'>внутри ipython ноутбука</font>):

- $S \rightarrow NP ~~ VP$
- $NP \rightarrow DET ~~ ADJ ~~ N \mid NN$
- $VP \rightarrow V ~~ NP \mid VP ~~ PP \mid V$
- $DET \rightarrow a$
- $ADJ \rightarrow tasty \mid ADV ~~ ADJ \mid \epsilon$
- $ADV \rightarrow very$
- $N \rightarrow fish \mid fork \mid dog \mid boy$
- $NN \rightarrow Mary \mid John$
- $V \rightarrow eats$
- $PP \rightarrow P ~~ NP \mid \epsilon$
- $P \rightarrow with$


## 2. CYK Parser

Реализуйте синтаксический парсер, работающий по методу [**CYK**](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%9A%D0%BE%D0%BA%D0%B0_%E2%80%94_%D0%AF%D0%BD%D0%B3%D0%B5%D1%80%D0%B0_%E2%80%94_%D0%9A%D0%B0%D1%81%D0%B0%D0%BC%D0%B8) и выполняющий функцию проверки принадлежности строки грамматике.

In [50]:
from itertools import product

class CYKParser:
    def __init__(self, rules):
        """
            `rules` - your grammar
        """
        ### YOUR CODE HERE
        self._rules = rules
        ### END OF YOUR CODE
    
    ### SOME ADDITION FUNCTIONS IF YOU NEED IT
    ###
    ### END YOUR ADDITIONAL FUNCTIONS

    def parse(self, in_tokens):
        """
            `in_tokens` - input sentence
            return True in case of sentence can be parsed, False otherwise
        """
        c = [[[] for i in range(len(in_tokens))] for j in range(len(in_tokens)) ]
        ### YOUR CODE HERE
        for j in range(len(in_tokens)):
            for key, values in self._rules.items():
                if in_tokens[j] in values:
                    c[j][j].append(key)
            for i in range(j - 1, -1, -1):
                for k in range(i + 1, j + 1):
                    for key, values in self._rules.items():
                        for value in values:
                            BC = value.split(" ")
                            if len(BC) != 2:
                                continue
                            [B, C] = BC
                            if B in c[i][k - 1] and C in c[k][j]:
                                c[i][j].append(key)
        if 'S' in c[0][len(in_tokens) - 1]:
            return True
        return False
            
        ### END OF YOUR CODE

Введите сюда полученную грамматику из первого пункта.

In [51]:
RULES = {
    'S': ['NP VP'],
    'NP': ['DET ADJN', 'DET N', 'Mary', 'John'],
    'VP': ['V NP', 'VP PP', 'eats'],
    'DET': ['a'],
    'ADJ': ['tasty', 'ADV ADJ', 'very'],
    'ADV': ['very'],
    'N': ['fish', 'fork', 'dog', 'boy'],
    'NN': ['Mary', 'John'],
    'V': ['eats'],
    'PP': ['P NP'],
    'P': ['with'],
    'ADJN': ['ADJ N']
}

Время проверить нашу грамматику.

In [52]:
correct_sentences = [
    'Mary eats a fish',
    'John eats a very very very tasty fish with a fork',
    'a dog eats a boy',
    'Mary eats John with a fork',
    'John eats a fork',
    'a very fish eats Mary'
]

not_correct_sentences = [
    'John',
    'Mary eats a eats',
    'boy eats dog',
    'John eats a very very very tasty fish with fork',
    'a Mary fork',
    'eats',
    'a'
]

parser = CYKParser(RULES)
print('Positve examples:')
print('-------------------------------------------------------------')
for sentence in correct_sentences:
    if parser.parse(sentence.split()):
        print('OK!', sentence)
    else:
        print('ERROR!', sentence)
print('*************************************************************')
print('Negative examples:')  
print('-------------------------------------------------------------')
for sentence in not_correct_sentences:
    if not parser.parse(sentence.split()):
        print('OK!', sentence)
    else:
        print('ERROR!', sentence)
print('*************************************************************')

Positve examples:
-------------------------------------------------------------
OK! Mary eats a fish
OK! John eats a very very very tasty fish with a fork
OK! a dog eats a boy
OK! Mary eats John with a fork
OK! John eats a fork
OK! a very fish eats Mary
*************************************************************
Negative examples:
-------------------------------------------------------------
OK! John
OK! Mary eats a eats
OK! boy eats dog
OK! John eats a very very very tasty fish with fork
OK! a Mary fork
OK! eats
OK! a
*************************************************************


## 3. Extended CYK Parser 

Модифицируйте парсер так, чтобы он возвращал цепочку правил, составляющих разбор.

In [99]:
from itertools import product
from collections import defaultdict

class CYKParser:
    def __init__(self, rules):
        """
            `rules` - your grammar
        """
        ### YOUR CODE HERE
        self._rules = rules
        ### END OF YOUR CODE
    
    ### SOME ADDITION FUNCTIONS IF YOU NEED IT
    ###
    ### END YOUR ADDITIONAL FUNCTIONS
    
    def dfs(self, key, i, j):
        if i == j:
            return [key + ' -> ' + self._path[i][j][key]]
        B, C, i, k, j = self._path[i][j][key]
#         print(B, C, i, k, j)
#         print(self.dfs(B, i, k -1) + [key + '->' + B + ',' + C] + self.dfs(C, k, j))
        return [key + ' -> ' + B + ' ' + C] + self.dfs(B, i, k -1) + self.dfs(C, k, j)

    def parse(self, in_tokens):
        """
            `in_tokens` - input sentence
            return True in case of sentence can be parsed, False otherwise
        """
        c = [[[] for i in range(len(in_tokens))] for j in range(len(in_tokens)) ]
        self._path = [[defaultdict(int) for i in range(len(in_tokens))] for j in range(len(in_tokens)) ]
        ### YOUR CODE HERE
        for j in range(len(in_tokens)):
            for key, values in self._rules.items():
                if in_tokens[j] in values:
                    c[j][j].append(key)
                    self._path[j][j][key] = in_tokens[j]
            for i in range(j - 1, -1, -1):
                for k in range(i + 1, j + 1):
                    for key, values in self._rules.items():
                        for value in values:
                            BC = value.split(" ")
                            if len(BC) != 2:
                                continue
                            [B, C] = BC
                            if B in c[i][k - 1] and C in c[k][j]:
                                c[i][j].append(key)
                                self._path[i][j][key] = (B, C, i, k, j)
        if 'S' in c[0][len(in_tokens) - 1]:
             return [self.dfs('S', 0, len(in_tokens) - 1)]
        return []
            
        ### END OF YOUR CODE

In [100]:
for sent in correct_sentences:
#     print(CYKParser(RULES).parse(sent.split()))
    for chain in CYKParser(RULES).parse(sent.split()):
        for rule in chain:
            print(rule)
        print()

S -> NP VP
NP -> Mary
VP -> V NP
V -> eats
NP -> DET N
DET -> a
N -> fish

S -> NP VP
NP -> John
VP -> VP PP
VP -> V NP
V -> eats
NP -> DET ADJN
DET -> a
ADJN -> ADJ N
ADJ -> ADV ADJ
ADV -> very
ADJ -> ADV ADJ
ADV -> very
ADJ -> ADV ADJ
ADV -> very
ADJ -> tasty
N -> fish
PP -> P NP
P -> with
NP -> DET N
DET -> a
N -> fork

S -> NP VP
NP -> DET N
DET -> a
N -> dog
VP -> V NP
V -> eats
NP -> DET N
DET -> a
N -> boy

S -> NP VP
NP -> Mary
VP -> VP PP
VP -> V NP
V -> eats
NP -> John
PP -> P NP
P -> with
NP -> DET N
DET -> a
N -> fork

S -> NP VP
NP -> John
VP -> V NP
V -> eats
NP -> DET N
DET -> a
N -> fork

S -> NP VP
NP -> DET ADJN
DET -> a
ADJN -> ADJ N
ADJ -> very
N -> fish
VP -> V NP
V -> eats
NP -> Mary



## Вместо заключения 

Вспомним про nltk и Earley парсер. Грамматики тут задаются следующим образом:

In [7]:
import nltk
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det ADJ N | NN
    VP -> V NP | VP PP | V
    Det -> 'a'
    ADJ -> 'tasty' | ADV ADJ | 
    ADV -> 'very'
    N -> 'fish' | 'fork' | 'dog' | 'boy'
    NN -> 'Mary' | 'John'
    V -> 'eats'
    PP -> P NP | 
    P -> 'with'
""")

Создадим парсер и применим к предложению.

In [ ]:
parser = nltk.EarleyChartParser(grammar)
trees = list(parser.parse('a very fish eats Mary'.split()))
for tree in trees:
    print(tree)

Чтобы увидеть сам разбор, достаточно установить параметр *trace* у парсера (по умолчанию trace=0).

In [ ]:
parser = nltk.EarleyChartParser(grammar, trace=1)
trees = parser.parse('Mary eats a fish'.split())
for tree in trees:
    print(tree)

In [ ]:
parser = nltk.EarleyChartParser(grammar, trace=2)
trees = parser.parse('John eats a very very very tasty fish with a fork'.split())
for tree in trees:
    print(tree)